In [6]:
import struct

def bmp_to_grayscale(input_file_path, output_file_path, input_bpp=24, output_bpp=8):
    if input_bpp not in (24, 32):
        raise ValueError("Unsupported input bits per pixel: {}. Only 24 and 32 bpp are supported.".format(input_bpp))
    if output_bpp != 8:
        raise ValueError("Unsupported output bits per pixel: {}. Only 8 bpp output is supported.".format(output_bpp))

    with open(input_file_path, "rb") as input_file:
        # Read file headers
        bmp_file_header = input_file.read(14)
        bmp_info_header = input_file.read(40)

        # Get image dimensions and data offset
        width, height = struct.unpack("<LL", bmp_info_header[4:12])
        data_offset = struct.unpack("<L", bmp_file_header[10:14])[0]

        # Calculate row padding for input image
        input_row_padding = (4 - ((width * input_bpp) // 8) % 4) % 4

        # Set file pointer to beginning of image data
        input_file.seek(data_offset)

        # Prepare output file
        with open(output_file_path, "wb") as output_file:
            # Write file headers
            output_info_header = bmp_info_header[:28] + struct.pack("<L", output_bpp) + bmp_info_header[32:]
            output_file_header = bmp_file_header
            output_file_header = output_file_header[:10] + struct.pack("<L", data_offset + 1024) + output_file_header[14:]
            output_file.write(output_file_header)
            output_file.write(output_info_header)

            # Write grayscale color palette
            for i in range(256):
                output_file.write(bytes([i, i, i, 0]))

            # Read and write image data
            for i in range(height):
                in_buf = input_file.read((width * input_bpp) // 8)
                out_buf = bytearray()
                for j in range(0, len(in_buf), input_bpp // 8):
                    # Convert RGB to grayscale using luminosity method
                    gray_value = int(0.3 * in_buf[j] + 0.59 * in_buf[j+1] + 0.11 * in_buf[j+2])
                    out_buf.append(gray_value)
                # Pad row to multiple of 4 bytes
                output_row_padding = b"\x00" * (4 - ((width * output_bpp) // 8) % 4) % 4
                output_file.write(out_buf + output_row_padding)

    print("Updating has come to the end successfully!")


# Example usage
bmp_to_grayscale("pict.bmp", "output.bmp", input_bpp=24, output_bpp=8)

TypeError: not all arguments converted during bytes formatting

In [8]:
!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 2.8 MB/s eta 0:00:00


In [12]:
import cv2
import sys

def convert(input_file, input_bpp, output_bpp):
    # Load input image
    img = cv2.imread(input_file, cv2.IMREAD_UNCHANGED)


    # Convert to black and white
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Scale output image to desired bpp
    output_depth = cv2.CV_8U if output_bpp == 8 else cv2.CV_16U
    gray_scaled = cv2.convertScaleAbs(gray, alpha=(65535.0/(2**output_bpp-1)), beta=0.0)

    # Save output image
    output_file = "output.bmp"
    cv2.imwrite(output_file, gray_scaled)

if __name__ == '__main__':
    convert("pict.bmp", 24, 8)

In [14]:
from PIL import Image

def convert_4bit_to_16bit(input_path, output_path):
  # Load the input image
  img = Image.open(input_path)
  # Convert to 16-bit mode with 5 bits per color channel and 1 alpha bit
  img16 = img.convert("RGB555")
  # Save the output image
  img16.save(output_path)

# Example usage
convert_4bit_to_16bit("pict.bmp", "output.bmp")

ValueError: conversion from RGB to RGB555 not supported

In [19]:
import numpy as np
import struct

def convert_rgb_to_rgb555(img):
  # Get the width and height of the image
  width, height = img.size
  # Convert the image to a numpy array of uint8 values
  arr = np.array(img, dtype=np.uint8)
  # Shift and mask the color channels to get 5 bits per channel
  r = (arr[:,:,0] >> 3) & 0x1F
  g = (arr[:,:,1] >> 3) & 0x1F
  b = (arr[:,:,2] >> 3) & 0x1F
  # Combine the color channels into a single uint16 value per pixel
  rgb555 = (r << 10) | (g << 5) | b
  # Flatten the array and pack it into bytes using little-endian format
  bytes = struct.pack("<%dH" % (width * height), *rgb555.flatten())

  return bytes

def add_bmp_header(bytes, width, height):
    # Calculate the file size in bytes
    file_size = len(bytes) + 54 # Header size is fixed at 54 bytes

    # Create a list of header fields in little-endian format

    header_fields = [
        ("B", "M"), # Signature: BM for bitmap files
        ("<I", file_size), # File size in bytes
        ("<H",0), # Reserved field: must be zero
        ("<H",0), # Reserved field: must be zero
        ("<I",54), # Offset to start of pixel data in bytes
        ("<I",40), # Size of BITMAPINFOHEADER structure in bytes
        ("<i",width), # Image width in pixels
        ("<i",height), # Image height in pixels
        ("<H",1), # Number of color planes: must be one
        ("<H",16), # Number of bits per pixel: here we use RGB555 mode
        ("<I",0), # Compression method: here we use BI_RGB which means no compression
        ("<I",len(bytes)),# Size of pixel data in bytes
        ("<i",3780),# Horizontal resolution in pixels per meter: here we use a default value
        ("<i",3780),# Vertical resolution in pixels per meter: here we use a default value
        ("<I",0),# Number of colors in palette: here we use zero which means all colors are used
        ("<I",0)# Number of important colors: here we use zero which means all colors are important
    ]

    header_bytes = b""

    for fmt,value in header_fields:
      header_bytes += struct.pack(fmt,value)

    return header_bytes + bytes

# Example usage
from PIL import Image

img = Image.open("pict.bmp")
bytes = convert_rgb_to_rgb555(img)
bytes_with_header = add_bmp_header(bytes,img.width,img.height)
with open("out.bmp","wb") as f:
    f.write(bytes_with_header)

error: required argument is not an integer

In [25]:
# Import PIL
from PIL import Image

# Open the 4-bit BMP image
img = Image.open("pict4bit.bmp")
print(img.mode)
# Convert it to 16-bit mode (5 bits per channel and 1 alpha bit)
img = img.convert("RGB")
img = np.array(img)
img = img.astype(np.uint16) # Change the data type to unsigned 16-bit integers
img = (img >> 3) | (img << 13) # Shift the bits according to the mode specification
img = Image.fromarray(img, "I;16")

# Save the converted image as a new file
img.save("image_16bit.png")

P
